In [1]:
!pip install pydobot2 pyserial

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: C:\Python313\python.exe -m pip install --upgrade pip


In [2]:
!pip install --upgrade pip

Defaulting to user installation because normal site-packages is not writeable
  Using cached pip-25.2-py3-none-any.whl.metadata (4.7 kB)
Using cached pip-25.2-py3-none-any.whl (1.8 MB)



[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: C:\Python313\python.exe -m pip install --upgrade pip
ERROR: To modify pip, please run the following command:
C:\Python313\python.exe -m pip install --upgrade pip


In [3]:
import time
import pydobot
from pydobot.dobot import MODE_PTP
import json

In [4]:
PORT = 'COM8'
RAISE_AMOUNT = 80  # mm to lift above pickup/drop
DROP_OFFSET = 2    # mm above original Z for safe drop

# Home position
HOME = (293.8, 0.0, 150.0, 0)

# Safe intermediate point between pick and drop
SAFE_POINT = (259.2, -29.5, 136.1, -6.5)

# Define pick and drop positions for 4 cubes
CUBES = [
    {'pickup': (276.7, -82.4, -44.2, -16.6),'drop': (277.6, 6.8, -41.8, 1.4)},
    {'pickup': (218.2, -83.3, -42.7, -20.9), 'drop': (216.3, 2.8, -41.6, 0.7)},
    {'pickup': (274.5, -24.2, -43.2, -5.0), 'drop': (273.8, 63.2, -44.0, 13.0)},
    {'pickup': (215.2, -24.4, -42.1, -6.5),  'drop': (217.4, 61.7, -43.1, 15.8)}
]

def move_with_raise(device, pos, drop_offset=0):
    """
    Raise Z by RAISE_AMOUNT, move down, pick/drop
    drop_offset: mm to add to Z for drop-off (positive = higher)
    """
    # Apply drop offset only to Z
    target_z = pos[2] + drop_offset

    # Raise above target position
    raised = (pos[0], pos[1], target_z + RAISE_AMOUNT, pos[3])
    device.move_to(*raised)
    time.sleep(1)

    # Move down to target position (with offset if dropping)
    target = (pos[0], pos[1], target_z, pos[3])
    device.move_to(*target)
    time.sleep(1)

try:
    device = Dobot(port=PORT)
    time.sleep(1)

    print("Moving to home position...")
    device.move_to(*HOME)
    time.sleep(1)

    # Forward pass: pick and drop cubes in order
    for cube in CUBES:
        print(f"Picking up cube at {cube['pickup']}")
        move_with_raise(device, cube['pickup'])
        print("Activating suction...")
        device.suck(True)
        time.sleep(0.5)

        print("Moving to safe point...")
        device.move_to(*SAFE_POINT)
        time.sleep(1)

        print(f"Moving to drop at {cube['drop']}")
        move_with_raise(device, cube['drop'], drop_offset=DROP_OFFSET)  # added offset
        print("Releasing suction...")
        device.suck(False)
        time.sleep(1)

    # Reverse pass: pick and drop cubes in reverse order
    for cube in reversed(CUBES):
        print(f"Picking up cube at {cube['drop']} for reverse move")
        move_with_raise(device, cube['drop'])
        print("Activating suction...")
        device.suck(True)
        time.sleep(1)

        print("Moving to safe point...")
        device.move_to(*SAFE_POINT)
        time.sleep(1)

        print(f"Returning cube to original pickup at {cube['pickup']}")
        move_with_raise(device, cube['pickup'], drop_offset=DROP_OFFSET)  # added offset
        print("Releasing suction...")
        device.suck(False)
        time.sleep(0.5)

    print("Returning to home position...")
    device.move_to(*HOME)
    time.sleep(1)

except Exception as e:
    print("Error:", e)

finally:
    if 'device' in locals() and device is not None:
        device.close()
        print("All tasks completed. Connection closed.")

Error: name 'Dobot' is not defined
